<a href="https://colab.research.google.com/github/Praneesh-Sharma/Celebal-Summer-Internship/blob/main/Assignment_Week_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-prices-advanced-regression-techniques:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5407%2F868283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240717%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240717T023949Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3647f6b195eeaebec05df8ac48133cc7a7fb48dcbabbeed243a03036b4d41ef39617a64df695412c71c04960792a20de5ea6843a1a5566284c3add3fcb2980eff492eefe0fb011726a796ca2fe8008cf90790399d3c9ef3347358e2ff5f42d46b7fa40a182493d350dd8c2e69c364e7db8dbe67218615e2190162df2b38e0bc99a6dc15c17ce7ad4148465457c980b9201b573fb91eb848e0956ac6f8ec03596f4530436b9ae1ca24cd4ef012c0dcb146f32dedeaeb2f95ec24ab149ec00fb2e2ca88c01d5c0ca6fd29bfb038f978f6e45f6770bf20cfcf454724daae1b51656935bd4cc3bd50ed2d89a04c778665568b84dbba7f07b16faa9c3026915fc2457'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 203809 bytes downloaded
Downloaded and uncompressed: house-prices-advanced-regression-techniques
Data source import complete.


## CSI Assignment Week_6

Author - Praneesh Sharma

### Importing Libraires

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from scipy import stats

### Loading Datasets

In [3]:
X = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
X_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

In [4]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
y = X['SalePrice']
X.drop('SalePrice', axis=1, inplace=True)

### Data Preprocessing

#### Handling MIssing Values

In [6]:
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(include=[object, 'category']).columns

In [7]:
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].mean())
X[categorical_cols] = X[categorical_cols].fillna(X[categorical_cols].mode().iloc[0])

In [8]:
X_test[numeric_cols] = X_test[numeric_cols].fillna(X_test[numeric_cols].mean())
X_test[categorical_cols] = X_test[categorical_cols].fillna(X_test[categorical_cols].mode().iloc[0])

#### Removing Outliers

In [9]:
z_scores = np.abs(stats.zscore(X[numeric_cols]))
X = X[(z_scores < 3).all(axis=1)]
y = y[X.index]

#### One Hot Encoding

In [10]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

In [11]:
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

#### Splitting the dataset

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#### Feature Scaling

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### Simple Linear Regression

In [14]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred_lr = linear_regressor.predict(X_val)
mse_lr = mean_squared_error(y_val, y_pred_lr)
print(f'Simple Linear Regression MSE: {mse_lr}')

Simple Linear Regression MSE: 8.510697387441032e+31


### Multiple Linear Regression

In [15]:
multiple_linear_regressor = LinearRegression()
multiple_linear_regressor.fit(X_train, y_train)
y_pred_mlr = multiple_linear_regressor.predict(X_val)
mse_mlr = mean_squared_error(y_val, y_pred_mlr)
print(f'Multiple Linear Regression MSE: {mse_mlr}')

Multiple Linear Regression MSE: 8.510697387441032e+31


### Polynomial Regression

In [16]:
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_val_poly = poly_features.transform(X_val)

poly_regressor = LinearRegression()
poly_regressor.fit(X_train_poly, y_train)
y_pred_poly = poly_regressor.predict(X_val_poly)
mse_poly = mean_squared_error(y_val, y_pred_poly)
print(f'Polynomial Regression MSE: {mse_poly}')

Polynomial Regression MSE: 1078128748.81834


### Ridge Regression

In [17]:
ridge_regressor = Ridge(alpha=1.0)
ridge_regressor.fit(X_train, y_train)
y_pred_ridge = ridge_regressor.predict(X_val)
mse_ridge = mean_squared_error(y_val, y_pred_ridge)
print(f'Ridge Regression MSE: {mse_ridge}')

Ridge Regression MSE: 553925850.0493609


### Lasso Regression

In [18]:
lasso_regressor = Lasso(alpha=1.0)
lasso_regressor.fit(X_train, y_train)
y_pred_lasso = lasso_regressor.predict(X_val)
mse_lasso = mean_squared_error(y_val, y_pred_lasso)
print(f'Lasso Regression MSE: {mse_lasso}')

Lasso Regression MSE: 557448147.76487


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.508e+09, tolerance: 3.501e+08
  model = cd_fast.enet_coordinate_descent(


### ElasticNet Regression

In [19]:
elastic_net_regressor = ElasticNet(alpha=1.0, l1_ratio=0.5)
elastic_net_regressor.fit(X_train, y_train)
y_pred_enet = elastic_net_regressor.predict(X_val)
mse_enet = mean_squared_error(y_val, y_pred_enet)
print(f'ElasticNet Regression MSE: {mse_enet}')

ElasticNet Regression MSE: 560659188.5146434
